# 🧠 Automatic Tumor Detection System
## BME 271D Final Project - Ege, Max, Sasha

### Using Frequency-Domain Analysis for Medical Image Segmentation

**What this tool does:**
1. Analyzes medical images using FFT (Fast Fourier Transform)
2. Applies frequency-domain filters to enhance tumor features
3. **Automatically detects if a tumor is present**
4. Highlights the tumor location
5. Compares multiple detection methods

In [ ]:
# ========== SETUP ==========
!pip install -q numpy matplotlib scipy scikit-image pandas

# Download our code from GitHub
!wget -q https://raw.githubusercontent.com/egeozemek/tumor-segmentation/main/tumor_segmentation.py
!wget -q https://raw.githubusercontent.com/egeozemek/tumor-segmentation/main/generate_realistic_tumors.py

# Download sample tumor images
!mkdir -p data/images data/masks
!wget -q -P data/images/ https://raw.githubusercontent.com/egeozemek/tumor-segmentation/main/data/images/tumor_001.png
!wget -q -P data/images/ https://raw.githubusercontent.com/egeozemek/tumor-segmentation/main/data/images/tumor_002.png
!wget -q -P data/images/ https://raw.githubusercontent.com/egeozemek/tumor-segmentation/main/data/images/tumor_003.png
!wget -q -P data/masks/ https://raw.githubusercontent.com/egeozemek/tumor-segmentation/main/data/masks/tumor_001.png
!wget -q -P data/masks/ https://raw.githubusercontent.com/egeozemek/tumor-segmentation/main/data/masks/tumor_002.png
!wget -q -P data/masks/ https://raw.githubusercontent.com/egeozemek/tumor-segmentation/main/data/masks/tumor_003.png

import tumor_segmentation as ts
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image as PILImage
from google.colab import files
from scipy import ndimage
from skimage.filters import threshold_otsu
from skimage.measure import label, regionprops
from skimage.morphology import remove_small_objects, binary_closing, binary_opening, disk

# ========== HELPER FUNCTIONS ==========
def load_image_safe(filepath):
    """Load any image and convert to grayscale float [0,1]"""
    img = PILImage.open(filepath).convert('L')
    arr = np.array(img).astype(np.float64) / 255.0
    return arr

def load_mask_safe(filepath):
    """Load any image and convert to binary mask"""
    img = PILImage.open(filepath).convert('L')
    arr = np.array(img)
    return (arr > 127).astype(np.uint8)

# ========== IMPROVED TUMOR DETECTION SYSTEM ==========
def detect_tumor_regions(image, sensitivity=0.5):
    """
    Detect potential tumor regions using multiple methods.
    Works for both bright tumors (real MRI) and dark tumors (synthetic).
    
    Returns dict with detection results for each method.
    """
    h, w = image.shape
    total_pixels = h * w
    
    # Adaptive thresholds based on sensitivity
    min_region_size = int(total_pixels * 0.002 * (1 - sensitivity * 0.8))  # Min 0.04% to 0.2%
    
    results = {}
    
    # ===== METHOD 1: Otsu on bright regions =====
    smoothed = ndimage.gaussian_filter(image, sigma=1.5)
    thresh = threshold_otsu(smoothed)
    
    # For real MRIs, tumors are often BRIGHTER
    bright_mask = smoothed > thresh
    bright_mask = binary_opening(bright_mask, disk(2))
    bright_mask = remove_small_objects(bright_mask, min_size=min_region_size)
    results['Baseline_Otsu'] = bright_mask
    
    # ===== METHOD 2: Adaptive threshold for high-intensity regions =====
    # Find regions significantly brighter than mean
    mean_val = image.mean()
    std_val = image.std()
    high_thresh = mean_val + (1.5 - sensitivity) * std_val
    bright_regions = image > high_thresh
    bright_regions = binary_closing(bright_regions, disk(3))
    bright_regions = remove_small_objects(bright_regions, min_size=min_region_size)
    results['Intensity_Threshold'] = bright_regions
    
    # ===== METHOD 3: FFT High-pass + threshold =====
    try:
        # High-pass to find edges/texture
        hp_img, _, _ = ts.filter_pipeline(image, 'hp', cutoff_radius=15)
        hp_img = (hp_img - hp_img.min()) / (hp_img.max() - hp_img.min() + 1e-8)
        
        # Find high-texture regions (tumors often have different texture)
        hp_thresh = np.percentile(hp_img, 70 - sensitivity * 20)
        hp_mask = hp_img > hp_thresh
        
        # Combine with intensity - tumor should be bright AND have texture
        combined_hp = np.logical_and(hp_mask, image > mean_val)
        combined_hp = binary_closing(combined_hp, disk(5))
        combined_hp = remove_small_objects(combined_hp, min_size=min_region_size)
        results['FFT_HighPass'] = combined_hp
    except:
        results['FFT_HighPass'] = np.zeros_like(image, dtype=bool)
    
    # ===== METHOD 4: FFT Band-pass =====
    try:
        bp_img, _, _ = ts.filter_pipeline(image, 'bp', r1=5, r2=40)
        bp_img = (bp_img - bp_img.min()) / (bp_img.max() - bp_img.min() + 1e-8)
        
        bp_thresh = np.percentile(bp_img, 75 - sensitivity * 25)
        bp_mask = bp_img > bp_thresh
        
        # Combine with intensity
        combined_bp = np.logical_and(bp_mask, image > mean_val)
        combined_bp = binary_closing(combined_bp, disk(5))
        combined_bp = remove_small_objects(combined_bp, min_size=min_region_size)
        results['FFT_BandPass'] = combined_bp
    except:
        results['FFT_BandPass'] = np.zeros_like(image, dtype=bool)
    
    # ===== METHOD 5: Combined voting =====
    # Region is suspicious if detected by multiple methods
    vote_map = (results['Baseline_Otsu'].astype(float) + 
                results['Intensity_Threshold'].astype(float) +
                results['FFT_HighPass'].astype(float) + 
                results['FFT_BandPass'].astype(float))
    
    # Need at least 2 methods to agree (or 1 at high sensitivity)
    vote_thresh = max(1, 2 - sensitivity)
    combined = vote_map >= vote_thresh
    combined = binary_closing(combined, disk(3))
    combined = remove_small_objects(combined, min_size=min_region_size)
    results['Combined'] = combined
    
    return results

def analyze_detection(image, mask):
    """
    Analyze a detected region and return metrics.
    """
    h, w = image.shape
    total_pixels = h * w
    
    tumor_pixels = mask.sum()
    tumor_area_pct = (tumor_pixels / total_pixels) * 100
    
    if tumor_pixels == 0:
        return {
            'detected': False,
            'area_percent': 0,
            'center': None,
            'confidence': 0
        }
    
    # Find center
    coords = np.where(mask)
    center = (int(np.mean(coords[0])), int(np.mean(coords[1])))
    
    # Calculate confidence based on region properties
    labeled = label(mask)
    regions = regionprops(labeled, intensity_image=image)
    
    if len(regions) == 0:
        return {'detected': False, 'area_percent': 0, 'center': None, 'confidence': 0}
    
    # Get largest region
    largest = max(regions, key=lambda x: x.area)
    
    # Confidence factors
    # 1. Size (not too small, not too big)
    size_score = min(tumor_area_pct / 3, 1.0) * min(1.0, 30 / (tumor_area_pct + 1))
    
    # 2. Compactness
    circularity = 4 * np.pi * largest.area / (largest.perimeter ** 2 + 1e-8)
    shape_score = min(circularity * 1.5, 1.0)
    
    # 3. Intensity contrast
    tumor_intensity = image[mask].mean()
    bg_intensity = image[~mask].mean() if (~mask).any() else 0
    contrast = abs(tumor_intensity - bg_intensity)
    contrast_score = min(contrast * 4, 1.0)
    
    confidence = (size_score * 0.3 + shape_score * 0.3 + contrast_score * 0.4)
    
    return {
        'detected': True,
        'area_percent': tumor_area_pct,
        'center': center,
        'confidence': np.clip(confidence, 0, 1)
    }

# Initialize
image = None
mask = None
print('✅ Setup complete! Tumor detection system ready.')

---
## 📁 Load Your Image

**Run ONE of the following cells:**

In [ ]:
# ========== OPTION A: UPLOAD YOUR OWN IMAGE ==========
print('Upload your medical image (MRI, CT, etc.):')

uploaded = files.upload()

if uploaded:
    uploaded_files = list(uploaded.keys())
    image = load_image_safe(uploaded_files[0])
    mask = None
    
    print(f'\n✅ Loaded: {uploaded_files[0]}')
    print(f'   Size: {image.shape[1]} x {image.shape[0]} pixels')
    
    plt.figure(figsize=(8, 8))
    plt.imshow(image, cmap='gray')
    plt.title('Your Uploaded Image')
    plt.axis('off')
    plt.show()
else:
    print('❌ No file uploaded.')

In [ ]:
# ========== OPTION B: USE SAMPLE IMAGES ==========
tumor_number = 1  # Change to 1, 2, or 3

tumor_file = f'tumor_{tumor_number:03d}.png'
print(f'Loading: {tumor_file}')

image = load_image_safe(f'data/images/{tumor_file}')
mask = load_mask_safe(f'data/masks/{tumor_file}')

print(f'✅ Loaded tumor {tumor_number} with ground truth')

fig, axes = plt.subplots(1, 2, figsize=(12, 5))
axes[0].imshow(image, cmap='gray')
axes[0].set_title('Sample Image')
axes[0].axis('off')
axes[1].imshow(image, cmap='gray')
axes[1].imshow(mask, cmap='Reds', alpha=0.5)
axes[1].set_title('Ground Truth')
axes[1].axis('off')
plt.show()

---
## 🔬 FFT Spectrum Analysis

In [ ]:
# ========== FFT ANALYSIS ==========
if image is not None:
    print('Computing FFT spectrum...')
    F_shift, mag = ts.compute_fft_spectrum(image)
    fig = ts.visualize_frequency_spectrum(image, F_shift)
    plt.show()
else:
    print('⚠️ No image loaded!')

---
## 🎯 TUMOR DETECTION & SEGMENTATION

Compare different detection methods

In [ ]:
# ========== MULTI-METHOD DETECTION ==========
if image is not None:
    print('Running tumor detection with multiple methods...\n')
    
    # Detect with sensitivity 0.5
    detection_results = detect_tumor_regions(image, sensitivity=0.5)
    
    # Create visualization - similar to your previous red/white style
    methods = ['Baseline_Otsu', 'Intensity_Threshold', 'FFT_HighPass', 'FFT_BandPass', 'Combined']
    
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.ravel()
    
    # Original image
    axes[0].imshow(image, cmap='gray')
    axes[0].set_title('Original Image', fontsize=12, fontweight='bold')
    axes[0].axis('off')
    
    # Each method
    for idx, method in enumerate(methods):
        detected_mask = detection_results[method]
        analysis = analyze_detection(image, detected_mask)
        
        axes[idx+1].imshow(image, cmap='gray')
        axes[idx+1].imshow(detected_mask, cmap='Reds', alpha=0.6)
        
        if analysis['center']:
            row, col = analysis['center']
            axes[idx+1].plot(col, row, 'g+', markersize=15, markeredgewidth=2)
        
        status = f"Area: {analysis['area_percent']:.1f}%" if analysis['detected'] else "Not detected"
        axes[idx+1].set_title(f'{method}\n{status}', fontsize=11, fontweight='bold')
        axes[idx+1].axis('off')
    
    plt.suptitle('Tumor Detection Method Comparison', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # Summary table
    print('\n' + '='*70)
    print(f'{"Method":<20} {"Detected":<12} {"Area %":<12} {"Confidence":<12}')
    print('='*70)
    for method in methods:
        analysis = analyze_detection(image, detection_results[method])
        det = '✓ YES' if analysis['detected'] else '✗ No'
        area = f"{analysis['area_percent']:.2f}%"
        conf = f"{analysis['confidence']*100:.0f}%" if analysis['detected'] else '-'
        print(f'{method:<20} {det:<12} {area:<12} {conf:<12}')
    print('='*70)
else:
    print('⚠️ No image loaded!')

---
## 🔍 AUTOMATIC TUMOR DETECTION VERDICT

In [ ]:
# ========== FINAL DETECTION RESULT ==========
if image is not None:
    # Use combined method for final result
    final_mask = detection_results['Combined']
    final_analysis = analyze_detection(image, final_mask)
    
    # Create nice visualization
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # Original
    axes[0].imshow(image, cmap='gray')
    axes[0].set_title('Original Image', fontsize=14, fontweight='bold')
    axes[0].axis('off')
    
    # Detection overlay
    axes[1].imshow(image, cmap='gray')
    if final_analysis['detected']:
        axes[1].imshow(final_mask, cmap='Reds', alpha=0.6)
        if final_analysis['center']:
            row, col = final_analysis['center']
            axes[1].plot(col, row, 'g+', markersize=25, markeredgewidth=3)
    axes[1].set_title('Detected Tumor Region', fontsize=14, fontweight='bold')
    axes[1].axis('off')
    
    # Results panel
    axes[2].axis('off')
    
    if final_analysis['detected']:
        if final_analysis['confidence'] > 0.6:
            status = '⚠️ TUMOR DETECTED'
            verdict = '🔴 HIGH CONFIDENCE'
        elif final_analysis['confidence'] > 0.3:
            status = '❓ POSSIBLE TUMOR'
            verdict = '🟡 MEDIUM CONFIDENCE'
        else:
            status = '❓ SUSPICIOUS REGION'
            verdict = '🟠 LOW CONFIDENCE'
    else:
        status = '✅ NO TUMOR DETECTED'
        verdict = '🟢 Image appears clear'
    
    results_text = f"""
    
    {status}
    
    ─────────────────────────────
    
    Confidence: {final_analysis['confidence']*100:.1f}%
    
    Tumor Area: {final_analysis['area_percent']:.2f}%
    
    {verdict}
    
    """
    
    axes[2].text(0.1, 0.5, results_text, fontsize=14, verticalalignment='center',
                fontfamily='monospace', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    axes[2].set_title('Detection Results', fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    print('\n' + '='*50)
    print(f'  {verdict}')
    print('='*50)
else:
    print('⚠️ No image loaded!')

---
## ⚙️ Adjust Sensitivity

In [ ]:
# ========== SENSITIVITY COMPARISON ==========
if image is not None:
    print('Comparing different sensitivity levels...\n')
    
    sensitivities = [0.2, 0.4, 0.6, 0.8]
    
    fig, axes = plt.subplots(1, 4, figsize=(16, 4))
    
    for idx, sens in enumerate(sensitivities):
        results = detect_tumor_regions(image, sensitivity=sens)
        combined_mask = results['Combined']
        analysis = analyze_detection(image, combined_mask)
        
        axes[idx].imshow(image, cmap='gray')
        if analysis['detected']:
            axes[idx].imshow(combined_mask, cmap='Reds', alpha=0.6)
        
        axes[idx].set_title(f'Sensitivity: {sens}\nArea: {analysis["area_percent"]:.1f}%', fontsize=11)
        axes[idx].axis('off')
    
    plt.suptitle('Effect of Detection Sensitivity', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    print('💡 Higher sensitivity = detects more/smaller regions')
    print('   Lower sensitivity = more conservative, fewer false positives')
else:
    print('⚠️ No image loaded!')

---
## 📊 Validation (if ground truth available)

In [ ]:
# ========== VALIDATION ==========
if image is not None and mask is not None:
    print('Validating detection against ground truth...\n')
    
    detected = detection_results['Combined']
    
    # Calculate metrics
    intersection = np.logical_and(detected, mask).sum()
    union = np.logical_or(detected, mask).sum()
    dice = 2 * intersection / (detected.sum() + mask.sum() + 1e-8)
    iou = intersection / (union + 1e-8)
    
    # Visualization
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    axes[0].imshow(image, cmap='gray')
    axes[0].imshow(mask, cmap='Greens', alpha=0.5)
    axes[0].set_title('Ground Truth', fontweight='bold')
    axes[0].axis('off')
    
    axes[1].imshow(image, cmap='gray')
    axes[1].imshow(detected, cmap='Reds', alpha=0.5)
    axes[1].set_title('Our Detection', fontweight='bold')
    axes[1].axis('off')
    
    # Overlap: Green=truth only, Red=detected only, Yellow=both
    overlap = np.zeros((*image.shape, 3))
    overlap[mask > 0] = [0, 1, 0]  # Green
    overlap[detected > 0] = [1, 0, 0]  # Red
    overlap[np.logical_and(mask, detected)] = [1, 1, 0]  # Yellow
    
    axes[2].imshow(image, cmap='gray')
    axes[2].imshow(overlap, alpha=0.6)
    axes[2].set_title(f'Overlap (Dice: {dice:.3f})', fontweight='bold')
    axes[2].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print(f'Dice Score: {dice:.3f}')
    print(f'IoU Score: {iou:.3f}')
elif mask is None:
    print('ℹ️ No ground truth mask available for validation.')
    print('   Upload an image with its corresponding mask to see validation metrics.')
else:
    print('⚠️ No image loaded!')